# Metagene plots
Now, let's draw **metagene plots**: (using seprate conda env and ipython kernel)

Using my `guitar` envirnment. 

In [1]:
%reload_ext rpy2.ipython

In [2]:
%%R
suppressMessages(suppressWarnings(library (Guitar)))

In [3]:
%%R 
txdb <- makeTxDbFromGFF('/rumi/shams/abe/genomes/hg38/gencode.v34/gencode.v34.annotation.gtf',organism='Homo sapiens')

R[write to console]: Import genomic features from the file as a GRanges object ... 
R[write to console]: OK

R[write to console]: Prepare the 'metadata' data frame ... 
R[write to console]: OK

R[write to console]: Make the TxDb object ... 
R[write to console]: OK



In [4]:
%%R 
GuitarPlot(txTxdb = txdb,stBedFiles = list("radar/result.sig.bed"),miscOutFilePrefix = "Guitar")

[1] "20210323213038"
[1] "There are 228048 transcripts of 60669 genes in the genome."
[1] "total 228048 transcripts extracted ..."
[1] "total 97941 transcripts left after ambiguity filter ..."
[1] "total 97941 transcripts left after check chromosome validity ..."
[1] "total 10939 mRNAs left after component length filter ..."
[1] "total 60598 ncRNAs left after ncRNA length filter ..."
[1] "generate components for all tx"
[1] "generate components for mRNA"
[1] "generate components for lncRNA"
[1] "generate chiped transcriptome"
[1] "generate coverage checking ranges for tx"
[1] "generate coverage checking ranges for mrna"
[1] "generate coverage checking ranges for ncrna"
[1] "20210323213529"
[1] "import BED file radar/result.sig.bed"
[1] "sample 10 points for Group1"
[1] "start figure plotting for tx ..."
[1] "start figure plotting for mrna ..."
[1] "start figure plotting for ncrna ..."


In [5]:
mv -v Guitar_* plots/

renamed 'Guitar_mrna_test.pdf' -> 'plots/Guitar_mrna_test.pdf'
renamed 'Guitar_ncrna_test.pdf' -> 'plots/Guitar_ncrna_test.pdf'
renamed 'Guitar_tx_test.pdf' -> 'plots/Guitar_tx_test.pdf'


In [6]:
!for f in plots/Guitar*.pdf; do bash ~/Workflows/my_scripts/pdf2png.sh $f; done

plots/Guitar_mrna_test.pdf > plots/Guitar_mrna_test.png
/rumi/shams/abe/Workflows/my_scripts/pdf2png.sh: line 5: magick: command not found
done!
plots/Guitar_ncrna_test.pdf > plots/Guitar_ncrna_test.png
/rumi/shams/abe/Workflows/my_scripts/pdf2png.sh: line 5: magick: command not found
done!
plots/Guitar_tx_test.pdf > plots/Guitar_tx_test.png
/rumi/shams/abe/Workflows/my_scripts/pdf2png.sh: line 5: magick: command not found
done!


# FIRE 
https://github.com/goodarzilab/FIRE
### Motif analysis 

#### Step 1: Extract mRNA exon sequences for significant hits from RADAR results 

In [1]:
%%bash 
# step 1: extract mRNA exon sequences for significant hits from RADAR 
bedtools getfasta -name -s -fi /rumi/shams/genomes/hg38/hg38.fa \
-bed radar/result.all.bed \
-split -fo radar/result.all.fa

#### Step 2: Prepare inputs for FIRE

Using my `bioseq` conda environment . 

In [2]:
# %%bash
# # step 2: prepare inputs for FIRE
# perl ${TEISERDIR}prep_seqs_for_teiser_run.pl radar/result.sig.fa radar/result.sig

!python $TEISERDIR/prep_fasta_for_fire_run.py radar/result.all.fa

#### Step 3: Run FIRE for known m6A motifs (non-discovery mode)

In [3]:
cat radar/known_motifs.txt

[AG]GAC
[AGT][AG]AC[ACT]


In [3]:
%%bash 
cd radar/
# step 3: run FIRE for known m6A motifs (non-discovery mode)
perl $FIREDIR/fire.pl \
    --expfile=result.sig_fire.txt \
    --exptype=discrete \
    --fastafile_rna=result.sig_fire.fa \
    --nodups=1 --dodna=0 --dodnarna=0 \
    --species=human \
    --doskipdiscovery=1 \
    --motiffile_rna=known_motifs.txt\
    --oribiasonly=0 #--jn_f=0

cd ../

mkdir -p fire 
rm -r fire/known_motifs_FIRE
mv -v radar/result.sig_fire.txt_FIRE/ fire/known_motifs_FIRE

Tue Mar 23 23:00:41 UTC 2021
RNA, Processing result.sig_fire.txt_FIRE/RNA/result.sig_fire.txt
RNA, Step 2: skip seed optimization, use -motiffile_rna option instead.
RNA, Step 3: evaluation of motif significances.
Processing motif [AG]GAC.
MI=0.00076809
Shuffle rank=17
Z-score=6.380
Sigificance category= NOT-SIGNIF
Robustness (jn_f=3)=0/10

Processing motif [AGT][AG]AC[ACT].
MI=0.00008237
Shuffle rank=3228
Z-score=0.039
Sigificance category= NOT-SIGNIF
Robustness (jn_f=3)=0/10

RNA, Step 3.5: creating binary expression profiles.
RNA, Step 4: discovery of distance constraints.
Read 2 motifs.
Read 9252 (genes) x 2 (conditions) matrix.
Read 9252 (genes) x 1 (conditions) matrix.
Reading sequences and building motif profiles ...Done.
Average sequence length is 50.
Quantized E vector into 2 bins
Motif [AG]GAC, present in 3240 genes.
       Average distance: MI=0.0014, rank=1711/10000, Z=0.8359, mbins=5
  Orientation bias {5'): MI=0.0008, rank=16/10000, Z=6.2295
  Orientation bias (3'): MI=0.

No motifs.
Can't use 'defined(@array)' (Maybe you should just omit the defined()?) at /flash/bin/FIRE-1.1/SCRIPTS/draw_position_histogram.pl line 134.
Table.pm: cannot open file "result.sig_fire.txt_FIRE/RNA/result.sig_fire.txt.clusters" ..
Table.pm: cannot open file "result.sig_fire.txt_FIRE/RNA/result.sig_fire.txt.mimatrix" ..
No motifs in @MOTIFS ...
No motifs in @MOTIFS ...
No motifs in @MOTIFS ...
No motifs in @MOTIFS ...
result.sig_fire.txt_FIRE/RNA/result.sig_fire.txt.fullmimatrix does not exists. at /flash/bin/FIRE-1.1/SCRIPTS/Sets.pm line 165.
result.sig_fire.txt_FIRE/RNA/result.sig_fire.txt.fullmimatrix does not exists. at /flash/bin/FIRE-1.1/SCRIPTS/Sets.pm line 165.


In [4]:
%%bash 
cd radar/
# step 3: run FIRE for known m6A motifs (non-discovery mode)
perl $FIREDIR/fire.pl \
    --expfile=result.all_fire.txt \
    --exptype=discrete \
    --fastafile_rna=result.all_fire.fa \
    --nodups=1 --dodna=0 --dodnarna=0 \
    --species=human \
    --doskipdiscovery=1 \
    --motiffile_rna=known_motifs.txt\
    --oribiasonly=0 #--jn_f=0

cd ../

mkdir -p fire 
rm -r fire/known_motifs_FIRE
mv -v radar/result.sig_fire.txt_FIRE/ fire/known_motifs_all_res_FIRE

Tue Mar 23 23:01:24 UTC 2021
RNA, Processing result.all_fire.txt_FIRE/RNA/result.all_fire.txt
RNA, Step 2: skip seed optimization, use -motiffile_rna option instead.
RNA, Step 3: evaluation of motif significances.
Processing motif [AG]GAC.
MI=0.00017159
Shuffle rank=0
Z-score=14.473
Sigificance category= OK-NO-SEED
Robustness (jn_f=3)=3/10

Processing motif [AGT][AG]AC[ACT].
MI=0.00000411
Shuffle rank=4747
Z-score=-0.336
Sigificance category= NOT-SIGNIF
Robustness (jn_f=3)=0/10

RNA, Step 3.5: creating binary expression profiles.
RNA, Step 4: discovery of distance constraints.
Read 2 motifs.
Read 91132 (genes) x 2 (conditions) matrix.
Read 91132 (genes) x 1 (conditions) matrix.
Reading sequences and building motif profiles ...Done.
Average sequence length is 118.
Quantized E vector into 2 bins
Motif [AG]GAC, present in 43454 genes.
       Average distance: MI=0.0003, rank=13/10000, Z=4.8676, mbins=5
  Orientation bias {5'): MI=0.0002, rank=0/10000, Z=14.5830
  Orientation bias (3'): MI

No motifs.
Can't use 'defined(@array)' (Maybe you should just omit the defined()?) at /flash/bin/FIRE-1.1/SCRIPTS/draw_position_histogram.pl line 134.
Table.pm: cannot open file "result.all_fire.txt_FIRE/RNA/result.all_fire.txt.clusters" ..
Table.pm: cannot open file "result.all_fire.txt_FIRE/RNA/result.all_fire.txt.mimatrix" ..
No motifs in @MOTIFS ...
No motifs in @MOTIFS ...
No motifs in @MOTIFS ...
No motifs in @MOTIFS ...
result.all_fire.txt_FIRE/RNA/result.all_fire.txt.fullmimatrix does not exists. at /flash/bin/FIRE-1.1/SCRIPTS/Sets.pm line 165.
result.all_fire.txt_FIRE/RNA/result.all_fire.txt.fullmimatrix does not exists. at /flash/bin/FIRE-1.1/SCRIPTS/Sets.pm line 165.
mv: cannot stat 'radar/result.sig_fire.txt_FIRE/': No such file or directory


CalledProcessError: Command 'b'cd radar/\n# step 3: run FIRE for known m6A motifs (non-discovery mode)\nperl $FIREDIR/fire.pl \\\n    --expfile=result.all_fire.txt \\\n    --exptype=discrete \\\n    --fastafile_rna=result.all_fire.fa \\\n    --nodups=1 --dodna=0 --dodnarna=0 \\\n    --species=human \\\n    --doskipdiscovery=1 \\\n    --motiffile_rna=known_motifs.txt\\\n    --oribiasonly=0 #--jn_f=0\n\ncd ../\n\nmkdir -p fire \nrm -r fire/known_motifs_FIRE\nmv -v radar/result.sig_fire.txt_FIRE/ fire/known_motifs_all_res_FIRE\n'' returned non-zero exit status 1.

#### Step 4: Run FIRE for all possible motifs (discovery mode)

In [1]:
%%bash 
cd radar/
# run FIRE --doskipdiscovery=0
perl $FIREDIR/fire.pl \
    --expfile=result.sig_fire.txt \
    --exptype=discrete \
    --fastafile_rna=result.sig_fire.fa \
    --nodups=1 --dodna=0 --dodnarna=0 \
    --species=human --oribiasonly=0

cd ../
rm -r fire/discovery_FIRE

mkdir -p fire 
mv -v radar/result.sig_fire.txt_FIRE/ fire/discovery_FIRE

Tue Mar 23 22:07:32 UTC 2021
RNA, Processing result.sig_fire.txt_FIRE/RNA/result.sig_fire.txt
RNA, Step 1: seed discovery.
Setting -shuffle to 16384.
Expression file is a 9252 by 1 matrix.
Min expression value read is 0, max is 1
Reading sequences ... Done
Number of ORFs = 9252
Calculating Mutual Information for all seeds ... Done
top 10 seeds...
AGAAACC
Determining threshold ...
Evaluating seed #0, AGAAACC ... passed.
Evaluating seed #100, CCCTCGA ... not passed.
Decreasing intervals phase.
Evaluating seed #50, AGAGAAA ... passed.
Evaluating seed #75, AGTCGAG ... not passed.
Evaluating seed #62, GTAATGA ... not passed.
Evaluating seed #56, CTTGACA ... not passed.
Searches for 5 consecutive 'not passed'.
Threshold set to seed #50 (included).
AGAAACC	0.0050
GAGAAAC	0.0046
GAATGTG	0.0046
AATGTGG	0.0039
CATACTG	0.0037
ATACTGG	0.0035
ACTGGAG	0.0030
CTGGAGA	0.0027
TGTGGCA	0.0026
TGAATGT	0.0026
TTCATAC	0.0024
GTGGGAA	0.0024
TCATACT	0.0024
CCCCCCT	0.0020
GTGGCAA	0.0018
TACAAAT	0.0018
GAAACCT	

corrupted size vs. prev_size
Aborted (core dumped)
free(): invalid next size (fast)
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
Segmentation fault (core dumped)
Segmentation fault (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
corrupted size vs. prev_size
Aborted (core dumped)
Can't use 'defined(@array)' (Maybe you should just omit the defined()?) at /flash/bin/FIRE-1.1/SCRIPTS/draw_position_histogram.pl line 134.
Can't locate Hypergeom.pm in @INC (you may need to install the Hypergeom module) (@INC contains: /nvme/bins/FIRE-1.1/SCRIPTS/../modules/lib/perl5/site_perl/5.12.4/darwin-thread-m

In [2]:
%%bash 
for f in  fire/*/RNA/*.pdf; do 
    o=${f/\/RNA\/result.sig_fire.txt/}
    mv -v $f $o; 
done 

renamed 'fire/discovery_FIRE/RNA/result.sig_fire.txt.densities.pdf' -> 'fire/discovery_FIRE.densities.pdf'
renamed 'fire/discovery_FIRE/RNA/result.sig_fire.txt.fullmimatrix.pdf' -> 'fire/discovery_FIRE.fullmimatrix.pdf'
renamed 'fire/discovery_FIRE/RNA/result.sig_fire.txt.summary.pdf' -> 'fire/discovery_FIRE.summary.pdf'


Using my `perl` env.

In [3]:
!for f in fire/*pdf; do bash /rumi/shams/abe/Workflows/my_scripts/pdf2png.sh $f; done

fire/discovery_FIRE.densities.pdf > fire/discovery_FIRE.densities.png
done!
fire/discovery_FIRE.fullmimatrix.pdf > fire/discovery_FIRE.fullmimatrix.png
done!
fire/discovery_FIRE.summary.pdf > fire/discovery_FIRE.summary.png
done!


### DRACH & RGAC motifs

<img src=fire/known_motifs_FIRE.png style="width:1100px">

## Discover over/under represented motifs 

<img src=fire/discovery_FIRE.summary.png style="width:1100px">

# iPAGE
https://github.com/goodarzilab/PAGE

In [4]:
%%bash 
ipage_cm='~/GitHub/Abe/my_scripts/ipage_human_ensembl.sh'
nohup ls hl60_delta_mtyl.txt | parallel -j18 -k bash $ipage_cm {} &> ipage.out &